# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'M'
DATA_FORMAT = 'delta'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: M
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-M-delta/
      Number of trips (dataset rows): 15,571,166
    


In [0]:
print_count(trips)

Row count: 15,571,166


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
142,43,2.1,1.0,11.8
238,151,0.2,1.0,4.3
132,165,14.7,1.0,51.95
138,132,10.6,0.0,36.35
68,33,4.94,1.0,24.36


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names)

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below


In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    df=df[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']]

    df= df.filter(~((df['trip_distance']==0) |
                  (df['passenger_count']==0) & (df['total_amount']==0)))

    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 15,350,075


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
142,43,2.1,1.0,11.8
238,151,0.2,1.0,4.3
132,165,14.7,1.0,51.95
138,132,10.6,0.0,36.35
68,33,4.94,1.0,24.36
224,68,1.6,1.0,14.15
95,157,4.1,1.0,17.3
90,40,5.7,1.0,21.8
97,129,9.1,1.0,28.8
263,142,2.7,2.0,18.95


In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    #for total amount
    median_totalamt=df.agg(pf.percentile_approx("total_amount", 0.5)).collect()[0][0]
    median_xi_totalamt= pf.abs(df['total_amount']-median_totalamt)
    MAD_totalamt=1.483*( df.agg(pf.percentile_approx(median_xi_totalamt,0.5)).collect()[0][0] )
    z_score_totalamt=(df['total_amount']-median_totalamt)/MAD_totalamt
    df=df.withColumn("z_score_totalAmount",z_score_totalamt)
    df=df.filter(pf.abs(df['z_score_totalAmount'])<3.5)  
    
    #for trip distance
    median_tripdist=df.agg(pf.percentile_approx("trip_distance", 0.5)).collect()[0][0]
    median_xi_tripdist= pf.abs(df['trip_distance']-median_tripdist)
    MAD_tripdist=1.483*( df.agg(pf.percentile_approx(median_xi_tripdist,0.5)).collect()[0][0] )
    z_score_tripdist=(df['trip_distance']-median_tripdist)/MAD_tripdist
    df=df.withColumn("z_score_tripDistance",z_score_totalamt)
    df=df.filter(pf.abs(df['z_score_tripDistance'])<3.5)
    
    df=df.drop(*('z_score_totalAmount','z_score_tripDistance'))
    
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)
print_count(trips_12)
display(trips_12.take(10))

Row count: 13,955,540


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
142,43,2.1,1.0,11.8
238,151,0.2,1.0,4.3
68,33,4.94,1.0,24.36
224,68,1.6,1.0,14.15
95,157,4.1,1.0,17.3
90,40,5.7,1.0,21.8
97,129,9.1,1.0,28.8
263,142,2.7,2.0,18.95
164,255,6.11,3.0,24.3
255,80,1.21,2.0,10.79


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below


In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    #Adding start zone name
    df=df.alias('t').join(zones_df,(df.PULocationID==zones_df.LocationID),'left').select('t.*',zones_df.Zone.alias("Start_zone")).alias('t1').join(zones_df.alias("z"),(df.DOLocationID==zones_df.LocationID),'left').select('t1.*',pf.col('z.Zone').alias("Stop_zone"))
    
    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
display(trips_21.take(10))

Row count: 13,955,540


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,Start_zone,Stop_zone
142,43,2.1,1.0,11.8,Lincoln Square East,Central Park
238,151,0.2,1.0,4.3,Upper West Side North,Manhattan Valley
68,33,4.94,1.0,24.36,East Chelsea,Brooklyn Heights
224,68,1.6,1.0,14.15,Stuy Town/Peter Cooper Village,East Chelsea
95,157,4.1,1.0,17.3,Forest Hills,Maspeth
90,40,5.7,1.0,21.8,Flatiron,Carroll Gardens
97,129,9.1,1.0,28.8,Fort Greene,Jackson Heights
263,142,2.7,2.0,18.95,Yorkville West,Lincoln Square East
164,255,6.11,3.0,24.3,Midtown South,Williamsburg (North Side)
255,80,1.21,2.0,10.79,Williamsburg (North Side),East Williamsburg


In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    #Adding unit profitability
    df=df.withColumn("unit_profitability",(df['total_amount']/df['trip_distance']))
    
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
display(trips_22.take(10))

Row count: 13,955,540


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,Start_zone,Stop_zone,unit_profitability
142,43,2.1,1.0,11.8,Lincoln Square East,Central Park,5.6190476190476195
238,151,0.2,1.0,4.3,Upper West Side North,Manhattan Valley,21.499999999999996
68,33,4.94,1.0,24.36,East Chelsea,Brooklyn Heights,4.931174089068826
224,68,1.6,1.0,14.15,Stuy Town/Peter Cooper Village,East Chelsea,8.84375
95,157,4.1,1.0,17.3,Forest Hills,Maspeth,4.219512195121951
90,40,5.7,1.0,21.8,Flatiron,Carroll Gardens,3.824561403508772
97,129,9.1,1.0,28.8,Fort Greene,Jackson Heights,3.164835164835165
263,142,2.7,2.0,18.95,Yorkville West,Lincoln Square East,7.018518518518518
164,255,6.11,3.0,24.3,Midtown South,Williamsburg (North Side),3.9770867430441896
255,80,1.21,2.0,10.79,Williamsburg (North Side),East Williamsburg,8.917355371900825


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below


In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    df=df.groupBy("PULocationID","DOLocationID").agg(pf.sum("passenger_count").alias("total_passengers"),
                                                                       pf.count("*").alias("trips_count"),
                                                                      pf.avg("unit_profitability").alias("average_unit_profit"))
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 23,737


PULocationID,DOLocationID,total_passengers,trips_count,average_unit_profit
97,129,23.0,18,3.2916901349447554
90,142,6986.0,5145,6.772791981961135
75,82,154.0,168,4.225238152881822
234,144,10457.0,7304,9.131209376748295
246,249,8354.0,5853,8.565721873043543
143,211,950.0,693,5.609656948246606
148,262,1468.0,1064,4.844512141795856
90,231,9329.0,6831,7.385969709180691
48,232,1759.0,1273,6.466584595272104
137,37,373.0,317,4.627472158010859


In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    
    #grouping by source zone
    df_agg=df.groupBy(df.PULocationID).agg(pf.sum("trips_count").alias("trips_count"),
                                        pf.sum("total_passengers").alias("total_passengers"),
                                        pf.avg("average_unit_profit").alias("average_unit_profit"))
    
    # Aliasing the dataframes
    df_agg = df_agg.alias("df_agg")
    zones_df = zones_df.alias("zones_df")

    # Joining the dataframes
    df_zone = df_agg.join(zones_df.alias("z"), df_agg['PULocationID'] == zones_df['LocationID'], "left") \
    .select(pf.col('z.Zone').alias("Start_Zone"),df_agg['*']).drop(df_agg['PULocationID'])
    
    return df_zone

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    return (df_zones.sort(df_zones.trips_count.desc())[['Start_Zone','trips_count']].take(10))

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    return (df_zones.sort(df_zones.average_unit_profit.desc())[['Start_Zone','average_unit_profit']].take(10))

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    return (df_zones.sort((df_zones.total_passengers/df_zones.trips_count).desc())[['Start_Zone',df_zones.total_passengers/df_zones.trips_count]].take(10) )

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)
display(zones)
top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

Start_Zone,trips_count,total_passengers,average_unit_profit
Brighton Beach,1148,174.0,8.525489246758692
Borough Park,3400,329.0,9.124777865690941
Downtown Brooklyn/MetroTech,19367,21221.0,6.2987226225409
Queens Village,1143,349.0,7.531470338751421
Starrett City,2251,76.0,16.77118519412257
Washington Heights North,5783,4902.0,8.523350265814049
Bellerose,464,83.0,7.229680604038302
Columbia Street,321,166.0,8.23725137757846
Greenpoint,2836,2177.0,9.34603623685534
Marine Park/Mill Basin,2587,170.0,7.421870406412921


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

Start_Zone,trips_count
Upper East Side South,757482
Upper East Side North,713787
Midtown Center,480172
Penn Station/Madison Sq West,472560
Lincoln Square East,458247
Upper West Side South,456141
Lenox Hill West,453406
Murray Hill,452439
Midtown East,430863
Union Sq,385269


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

Start_Zone,average_unit_profit
Mariners Harbor,119.92820010227994
Rossville/Woodrow,111.5625
West Brighton,87.37120340543028
Newark Airport,53.58419177850159
Stapleton,48.509328316674505
Van Cortlandt Village,43.1432908274593
Prospect Park,38.650704729994246
Rosedale,35.40380467503686
Bay Terrace/Fort Totten,35.07682279901328
Sheepshead Bay,34.099339613447704


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

Start_Zone,(total_passengers / trips_count)
Oakwood,3.5
Broad Channel,2.5714285714285716
Jamaica Bay,2.0
Great Kills,1.6666666666666667
Battery Park,1.6612458654906284
Rikers Island,1.5
West Brighton,1.5
Central Park,1.4922613202694779
JFK Airport,1.484916040415663
World Trade Center,1.4749132611721687


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XL'
DATA_FORMAT = 'parquet'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: XL
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_1_6_2019.parquet', '/FileStore/tables/taxi/parquet/tripdata_7_12_2019.parquet']
      Number of trips (dataset rows): 90,443,069
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[134]: [[Row(Start_Zone='Upper East Side South', trips_count=3630690),
  Row(Start_Zone='Midtown Center', trips_count=3436227),
  Row(Start_Zone='Upper East Side North', trips_count=3294361),
  Row(Start_Zone='Midtown East', trips_count=3033936),
  Row(Start_Zone='Penn Station/Madison Sq West', trips_count=3014037),
  Row(Start_Zone='Times Sq/Theatre District', trips_count=2883222),
  Row(Start_Zone='JFK Airport', trips_count=2670657),
  Row(Start_Zone='Clinton East', trips_count=2663452),
  Row(Start_Zone='Murray Hill', trips_count=2591587),
  Row(Start_Zone='Union Sq', trips_count=2525485)],
 [Row(Start_Zone='Newark Airport', average_unit_profit=92.59463830445044),
  Row(Start_Zone='Rossville/Woodrow', average_unit_profit=74.59719904896612),
  Row(Start_Zone='Arden Heights', average_unit_profit=44.875260495056615),
  Row(Start_Zone="Eltingville/Annadale/Prince's Bay", average_unit_profit=35.86964022870517),
  Row(Start_Zone='NV', average_unit_profit=23.224590746721965),
  Row(Star

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.898M |  15.57M |  41.954M |  90.443M |  132.396M |
| execution time   (w/o 1.2)  | 6.45s | 1m21s | 13m42s | 17m09s | 19m56s |
| execution time              | 15.02s | 2m52s | 27m23s| 40m27s | 46m52s |
| sec / 1M records (w/o 1.2)  |  2.225 | 5.2  |  19.6 | 11.37  | 9.03 |
| sec / 1M records            | 5.18  | 11.04  | 39.16  |  26.83 | 21.023  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.898M |  15.57M |  41.954M |  90.443M |  132.396M |
| execution time   (w/o 1.2)  | 5.7s | 8.03s | 8.11s | 14.77s | 31.62s |
| execution time              | 10.55s | 18.61s | 19.82s | 32.64s | 46.48s |
| sec / 1M records (w/o 1.2)  | 1.968  | 0.516  | 0.19  | 0.163  | 0.239  |
| sec / 1M records            | 3.64  | 1.195  | 0.472  | 0.36  | 0.35  |